In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct",
    torch_dtype="auto",
    attn_implementation="flash_attention_2",
    device_map="auto",
)

processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", use_fast=True)

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [ ]:
import base64
from io import BytesIO
import os
from decord import VideoReader
from PIL import Image
import numpy as np

import sys
sys.path.append("/projects/cft_vlm/finetune")
from qwenvl.common_utils import make_model_input
processor.tokenizer.padding_side = "right"
message = [
  [
      {
          "role": "system",
          "content": "You are a student in medicine studying for a final exam."
      },
      {
          "role": "user",
          "content": [
              {"type": "text", "text": "Describe this video"},
              {"type": "video", "video": "/projects/cft_vlm/openbiomedvid/vid_processed/-0WrCtDe6IQ_8_567.mp4"},
          ]
      }
  ],
  [
      {
          "role": "system",
          "content": "You are a student in medicine studying for a final exam."
      },
      {
          "role": "user",
          "content": [
              {"type": "text", "text": "Describe this video"},
              {"type": "video", "video": "/projects/cft_vlm/openbiomedvid/vid_processed/-0zSOhy4mgo_1220_1265.mp4"}
          ]
      }
  ],  
]

inputs = make_model_input(
    message[0],
    processor,
    base_interval=1,
    video_min_frames=8,
    video_max_frames=16
)

Unused or unrecognized kwargs: fps, return_tensors.


In [14]:
from qwenvl.common_utils import make_model_input

base_interval = 1
video_min_frames = 8
video_max_frames = 16
conversations = message

data_dict = make_model_input(
    conversations,
    processor,
    base_interval=base_interval,
    video_min_frames=video_min_frames,
    video_max_frames=video_max_frames,
    add_generation_prompt=True,
)
out = model.generate(**data_dict, max_new_tokens=30, do_sample=False)

out_txt = processor.tokenizer.batch_decode(out, skip_special_tokens=True)
out_txt

Unused or unrecognized kwargs: fps, return_tensors.
Unused or unrecognized kwargs: fps, return_tensors.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/fs01/projects/cft_vlm/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:2479: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


['system\nYou are a student in medicine studying for a final exam.\nuser\nDescribe this video\nassistant\nThe video appears to be an educational resource, likely used for medical students or professionals to learn about various liver conditions and their ultrasound appearances. The content is',
 'system\nYou are a student in medicine studying for a final exam.\nuser\nDescribe this video\nassistant\nThis image appears to be a slide from a medical presentation, specifically focusing on the clinical characteristics of a condition related to the eye. The title "Clinical']

In [15]:
data_dict.keys()

dict_keys(['input_ids', 'attention_mask', 'pixel_values_videos', 'video_grid_thw', 'second_per_grid_ts', 'labels'])

In [27]:
out2 = model.generate(**inputs, max_new_tokens=30, do_sample=False)
out2 = processor.tokenizer.batch_decode(out2, skip_special_tokens=True)
out2

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/fs01/projects/cft_vlm/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:2479: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


['system\nYou are a helpful assistant.\nuser\nDescribe this video\nassistant\nThe provided images are ultrasound scans of the liver, showing various normal and abnormal conditions. Here is a detailed description of each image:\n\n1. **Normal']